# Machine Learning II

## Laboratorio práctico de Regresión Lineal, pt. 2

### Trabajando con variables dummies

Trabajaremos en base a un ejercicio que fue propuesto en un examen de Análisis de Datos para otro Mg en Data Science. El ejercicio ha sido simplificado en ciertas especificaciones (no se pide anaizar heterocedasticidad, por ejemplo, ni estimar con estimadores robustos de la heterocedasticidad)

En el siguiente URL: https://bit.ly/3l6nHri usted dispone de la BD de la encuesta CASEN 2009. Consiste en un archivo Excel y la documentación necesaria para entender las columnas.

P1) Suponga que se quiere estimar un modelo para explicar los ingresos por trabajo, a través del modelo:

$$\text{ingreso} = \beta_0+\beta_1\cdot \text{escolaridad}+\beta_2\cdot \text{experiencia}+\beta_3\cdot\text{experiencia}^2$$

Interpretar resultados al ajustar por mínimos cuadrados ordinarios

P2) Ahora agregue al modelo una variable dicotómica por sexo y un conjunto de variables dicotómicas
por región. Defina la Región Metropolitana como la categoría base. Restrinja la muestra a personas
menores de 65 años si son hombre y 60 si son mujeres (es decir, que no estén en edad de jubilar).
Además, considere sólo a trabajadores con contrato y que trabajen jornada completa. Interprete las nuevas estimaciones.


#### Solución

P1)  Como la variable “experiencia” no está en el modelo, debemos construirla a partir de los datos que
tenemos. Un supuesto plausible es considerar la experiencia como la edad, menos la escolaridad, menos
6, que es cuando partimos la escolaridad. Pues claro, la experiencia laboral por lo general parte desde que
se termina la formación, hasta el momento presente de la toma del dato. Ciertamente, también puede
considerarse como la experiencia definida como la edad menos la escolaridad. Si bien hay gente que parte
trabajando antes de egresar, un supuesto es que ese conjunto es una minoría, y no tenemos el dato duro
que muestra esa proporción. Por parsimonia, nos quedamos con este supuesto.

Además, se deben hacer limpiezas a los datos para poder trabajar con ellos y hacer una regresión. Se hace:



In [6]:
import pandas as pd
import os

os.chdir('C:/Users/aleph/OneDrive/Escritorio/UDLA Mg DS/datasets/CASEN')
df = pd.read_csv('CASEN 2009 con ingresos orig y ajust.csv', sep=',', header=0, encoding='latin-1')

df

C:\Users\aleph\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (52,214,249,253,302,303,304,305,306,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,560,562,564,566,568,570,576) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,SEGMENTO,IDVIV,HOGAR,O,FOLIO,REGION,PROVINCIA,COMUNA,ZONA,ESTRATO,...,Y21E3X,Y21M4,Y21I4,Y21E4X,Y21M5,Y21I5,Y21E5X,Y21N6X,Y21M6,Y21I6
0,1101101,1,1,1,100011-K,1,11,1101,1,11011,...,,0,,,0,,,,0,
1,1101101,1,1,2,100011-K,1,11,1101,1,11011,...,,0,,,0,,,,0,
2,1101101,1,1,3,100011-K,1,11,1101,1,11011,...,,0,,,0,,,,0,
3,1101101,4,1,1,100041-1,1,11,1101,1,11011,...,,0,,,0,,,,0,
4,1101101,4,1,2,100041-1,1,11,1101,1,11011,...,,0,,,0,,,,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246919,15202202,48,1,1,916361-1,15,152,15202,2,152022,...,,0,,,0,,,,0,
246920,15202202,49,1,1,916371-9,15,152,15202,2,152022,...,,0,,,0,,,,0,
246921,15202202,50,1,1,916381-6,15,152,15202,2,152022,...,,0,,,0,,,,0,
246922,15202202,54,1,1,916421-9,15,152,15202,2,152022,...,,0,,,0,,,,0,


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246924 entries, 0 to 246923
Columns: 604 entries, SEGMENTO to Y21I6
dtypes: int64(120), object(484)
memory usage: 1.1+ GB


In [8]:
# filtramos las columnas con las que trabajaremos primero
df2 = df[['YTRABAJ','ESC','EDAD']]

#limpieza de datos
df2['ESC'] = [x.replace(' ','') for x in df2['ESC']] #intente ejecutar la sgte linea sin esta
df2['ESC'] = pd.to_numeric(df2['ESC'])

df2['YTRABAJ'] = [x.replace(' ','') for x in df2['YTRABAJ']] #intente ejecutar la sgte linea sin esta
df2['YTRABAJ'] = pd.to_numeric(df2['YTRABAJ'])

## creacion de variables 
df2['EXP'] = df2['EDAD'] - df2['ESC'] - 6 
df2['EXP2'] = df2['EXP']*df2['EXP']

## quitar NA

df2.dropna(inplace=True)
df2

C:\Users\aleph\AppData\Local\Temp/ipykernel_10208/2356343771.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['ESC'] = [x.replace(' ','') for x in df2['ESC']] #intente ejecutar la sgte linea son esta
C:\Users\aleph\AppData\Local\Temp/ipykernel_10208/2356343771.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['ESC'] = pd.to_numeric(df2['ESC'])
C:\Users\aleph\AppData\Local\Temp/ipykernel_10208/2356343771.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

,YTRABAJ,ESC,EDAD,EXP,EXP2
3,369720.0,0.0,35,29.0,841.0
7,198900.0,5.0,49,38.0,1444.0
9,359450.0,7.0,39,26.0,676.0
14,442000.0,13.0,43,24.0,576.0
15,179653.0,12.0,30,12.0,144.0
...,...,...,...,...,...
246913,585390.0,6.0,60,48.0,2304.0
246914,229364.0,6.0,58,46.0,2116.0
246915,188284.0,2.0,52,44.0,1936.0
246918,154050.0,3.0,72,63.0,3969.0


In [9]:
# definición de matrices para el modelo
X = df2[['ESC', 'EXP','EXP2']]
Y = df2[['YTRABAJ']]

In [ ]:
# modelamiento

import statsmodels.api as sm
X = sm.add_constant(X)
X

C:\Users\aleph\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


,const,ESC,EXP,EXP2
3,1.0,0.0,29.0,841.0
7,1.0,5.0,38.0,1444.0
9,1.0,7.0,26.0,676.0
14,1.0,13.0,24.0,576.0
15,1.0,12.0,12.0,144.0
...,...,...,...,...
246913,1.0,6.0,48.0,2304.0
246914,1.0,6.0,46.0,2116.0
246915,1.0,2.0,44.0,1936.0
246918,1.0,3.0,63.0,3969.0


In [42]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96226 entries, 3 to 246921
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   const   96226 non-null  float64
 1   ESC     96226 non-null  float64
 2   EXP     96226 non-null  float64
 3   EXP2    96226 non-null  float64
dtypes: float64(4)
memory usage: 3.7 MB


Explicación: el fundamento está en la multiplicación de matrices. La matriz de arriba representan los valores de los atributos, entonces, en el modelo esta matriz multiplica a la matriz de coeficientes: 

$$Y=X\beta$$

en donde $\beta$ es el vector de todos los coeficientes ( $\beta = [\beta_0,\beta_1,\beta_2,\beta_3]$)

Una columna de puros 1 se corresponderá con el primer coeficiente, es decir, $\beta_0$

In [12]:
modelo = sm.OLS(Y,X.astype(float)) #estimacion MiCO
ajuste = modelo.fit()

In [13]:
ajuste.summary() #ver tabla de resultados

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                YTRABAJ   R-squared:                       0.096
Model:                            OLS   Adj. R-squared:                  0.096
Method:                 Least Squares   F-statistic:                     3397.
Date:                Sat, 22 Jul 2023   Prob (F-statistic):               0.00
Time:                        10:26:56   Log-Likelihood:            -1.3981e+06
No. Observations:               96226   AIC:                         2.796e+06
Df Residuals:                   96222   BIC:                         2.796e+06
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3.551e+05   7660.247    -46.360      0.000    -3.7e+05    -3.4e+05
ESC         4.638e+04    490.080     94.646      0.000    4.54e+04    4.73e+04
EXP         1.261e+04    296.345     42.542      0.000     1.2e+04    1.32e+04
EXP2        -102.4724      4.526    -22.642      0.000    -111.343     -93.602
==============================================================================
Omnibus:                   174609.307   Durbin-Watson:                   1.694
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        452444359.892
Skew:                          13.168   Prob(JB):                         0.00
Kurtosis:                     337.891   Cond. No.                     7.20e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.2e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Todas las variables resultaron ser significativas, aunque se presume que hay multicolinealidad. Los signos
corresponden a la intuición: entre más escolaridad y experiencia se tiene, mayores ingresos vamos a tener.
Por ejemplo, de acuerdo a los resultados, un año más de experiencia, significa 12600 pesos más de
ingreso. El signo negativo en la experiencia, indica que cuando somos más viejos, ya no nos va tan bien.

P2) Se debe filtrar el dataframe original, incorporando las columnas para el filtro, esto es, considerar O25 que habla se si tiene o no contrato, y O27 el tipo de jornada, ademas de SEXO y REGION.



In [50]:
df3 = df[['YTRABAJ','ESC','EDAD', 'SEXO', 'REGION','O25','O27']]

df4 = df3[(((df3.SEXO==1) & (df3.EDAD<65)) | ((df3.SEXO==2) & (df3.EDAD<60))) &
         ((df3.O25=='1')|(df3.O25=='2')) & (df3.O27=='1')]


In [51]:
# ahora viene la fase de generar dummies y de "Botar" una categoria base

dummies_sexo = pd.get_dummies(df4.SEXO)
dummies_region = pd.get_dummies(df4.REGION)


In [52]:
dummies_sexo

,1,2
7,0,1
14,1,0
15,0,1
28,0,1
33,1,0
...,...,...
246877,1,0
246892,1,0
246894,0,1
246898,1,0


In [53]:
dummies_region

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
7,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
15,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
28,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
33,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246877,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
246892,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
246894,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
246898,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [54]:
dummies_sexo.drop(1, axis=1, inplace=True) #botamos "hombre"
dummies_region.drop(13, axis=1, inplace=True) #botamos a la región metropilitana (13)

In [55]:
dummies_sexo

,2
7,1
14,0
15,1
28,1
33,0
...,...
246877,0
246892,0
246894,1
246898,0


In [56]:
dummies_region

,1,2,3,4,5,6,7,8,9,10,11,12,14,15
7,1,0,0,0,0,0,0,0,0,0,0,0,0,0
14,1,0,0,0,0,0,0,0,0,0,0,0,0,0
15,1,0,0,0,0,0,0,0,0,0,0,0,0,0
28,1,0,0,0,0,0,0,0,0,0,0,0,0,0
33,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246877,0,0,0,0,0,0,0,0,0,0,0,0,0,1
246892,0,0,0,0,0,0,0,0,0,0,0,0,0,1
246894,0,0,0,0,0,0,0,0,0,0,0,0,0,1
246898,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [57]:

#limpieza de datos
df4['ESC'] = [x.replace(' ','') for x in df4['ESC']] #intente ejecutar la sgte linea son esta
df4['ESC'] = pd.to_numeric(df4['ESC'])

df4['YTRABAJ'] = [x.replace(' ','') for x in df4['YTRABAJ']] #intente ejecutar la sgte linea son esta
df4['YTRABAJ'] = pd.to_numeric(df4['YTRABAJ'])

## creacion de variables 
df4['EXP'] = df4['EDAD'] - df4['ESC'] - 6 
df4['EXP2'] = df4['EXP']*df4['EXP']

#X2 = df4[['ESC', 'EXP','EXP2']]
df5 = pd.concat([df4,dummies_sexo], axis=1) 
df5 = pd.concat([df5,dummies_region], axis=1)

# drop NA antes de separar en X e Y
df5.dropna(inplace=True)
df5

C:\Users\aleph\AppData\Local\Temp/ipykernel_10208/3558647601.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['ESC'] = [x.replace(' ','') for x in df4['ESC']] #intente ejecutar la sgte linea son esta
C:\Users\aleph\AppData\Local\Temp/ipykernel_10208/3558647601.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['ESC'] = pd.to_numeric(df4['ESC'])
C:\Users\aleph\AppData\Local\Temp/ipykernel_10208/3558647601.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

,YTRABAJ,ESC,EDAD,SEXO,REGION,O25,O27,EXP,EXP2,2,...,5,6,7,8,9,10,11,12,14,15
7,198900.0,5.0,49,2,1,1,1,38.0,1444.0,1,...,0,0,0,0,0,0,0,0,0,0
14,442000.0,13.0,43,1,1,1,1,24.0,576.0,0,...,0,0,0,0,0,0,0,0,0,0
15,179653.0,12.0,30,2,1,1,1,12.0,144.0,1,...,0,0,0,0,0,0,0,0,0,0
28,187850.0,12.0,38,2,1,1,1,20.0,400.0,1,...,0,0,0,0,0,0,0,0,0,0
33,221000.0,7.0,55,1,1,1,1,42.0,1764.0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246877,255992.0,9.0,53,1,15,1,1,38.0,1444.0,0,...,0,0,0,0,0,0,0,0,0,1
246892,211229.0,12.0,28,1,15,1,1,10.0,100.0,0,...,0,0,0,0,0,0,0,0,0,1
246894,299823.0,5.0,25,2,15,1,1,14.0,196.0,1,...,0,0,0,0,0,0,0,0,0,1
246898,333802.0,12.0,42,1,15,1,1,24.0,576.0,0,...,0,0,0,0,0,0,0,0,0,1


In [58]:
Y2= df5[['YTRABAJ']]
X2= df5.drop(['YTRABAJ', 'EDAD', 'SEXO','REGION','O25','O27'], axis=1)


In [59]:
X2=sm.add_constant(X2)
modelo2 = sm.OLS(Y2,X2.astype(float)) #estimacion MiCO
ajuste2 = modelo2.fit()
ajuste2.summary()

C:\Users\aleph\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                YTRABAJ   R-squared:                       0.218
Model:                            OLS   Adj. R-squared:                  0.218
Method:                 Least Squares   F-statistic:                     682.7
Date:                Sat, 22 Jul 2023   Prob (F-statistic):               0.00
Time:                        12:36:23   Log-Likelihood:            -6.1547e+05
No. Observations:               44043   AIC:                         1.231e+06
Df Residuals:                   44024   BIC:                         1.231e+06
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -2.256e+05   7411.217    -30.435      0.000    -2.4e+05   -2.11e+05
ESC         4.405e+04    445.214     98.941      0.000    4.32e+04    4.49e+04
EXP         7204.7832    359.223     20.057      0.000    6500.700    7908.866
EXP2         -32.0030      7.513     -4.260      0.000     -46.729     -17.277
2          -9.617e+04   2927.590    -32.851      0.000   -1.02e+05   -9.04e+04
1          -1.138e+04   1.18e+04     -0.961      0.337   -3.46e+04    1.18e+04
2           6.238e+04   8021.750      7.777      0.000    4.67e+04    7.81e+04
3           2.986e+04   8960.786      3.332      0.001    1.23e+04    4.74e+04
4          -4.807e+04   7138.613     -6.734      0.000   -6.21e+04   -3.41e+04
5          -2.174e+04   4820.051     -4.509      0.000   -3.12e+04   -1.23e+04
6          -5.508e+04   4862.261    -11.329      0.000   -6.46e+04   -4.56e+04
7           -5.59e+04   5373.333    -10.402      0.000   -6.64e+04   -4.54e+04
8          -5.268e+04   4517.295    -11.662      0.000   -6.15e+04   -4.38e+04
9          -6.805e+04   6364.512    -10.692      0.000   -8.05e+04   -5.56e+04
10         -9774.1964   6011.212     -1.626      0.104   -2.16e+04    2007.887
11          5.687e+04   1.28e+04      4.431      0.000    3.17e+04     8.2e+04
12          1.012e+05    1.3e+04      7.803      0.000    7.57e+04    1.27e+05
14          -3.57e+04   8557.377     -4.172      0.000   -5.25e+04   -1.89e+04
15         -4.183e+04   1.57e+04     -2.658      0.008   -7.27e+04    -1.1e+04
==============================================================================
Omnibus:                    55785.868   Durbin-Watson:                   1.546
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         16976791.714
Skew:                           6.788   Prob(JB):                         0.00
Kurtosis:                      98.219   Cond. No.                     1.08e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.08e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Las variables originales tienen la misma interpretación. El coeficiente asociado al primer “2” arriba es el
coeficiente asociado a la dummie de SEXO=2, es decir, esto implica que el sueldo de las mujeres es
96170 pesos menor que el de los hombres. Para las regiones, se descartan del modelo la región de
Tarapacá y la de Los Lagos. En cuanto a los coeficientes, podemos decir que en las regiones de
Antofagasta, Atacama, Aysen y Magallanes, el sueldo es mejor que en la Región metropolitana

**Ejercicio**

- Terminar el modelamiento quitando las variables no significativas y concluir.